In [ ]:
import pandas as pd
import docx2txt
import re

In [2]:
text = docx2txt.process('2016 CCS - Leadership Survey - Final Questionnaire.docx')

In [3]:
text = text.splitlines()

In [4]:
#text

In [5]:
def is_valid(txt):
    pattern1_invalid = re.compile('(N*Q[0-9]+_*[A-Za-z]*)(.*)')
    pattern2_invalid = re.compile('(NA-[0-9]+)(.)')
    pattern3_invalid = re.compile('Display This Question:')
    pattern4_invalid = re.compile('Multilateral')
    if pattern1_invalid.match(txt):
        return False
    elif pattern2_invalid.match(txt):
        return False
    elif pattern3_invalid.match(txt):
        return False
    elif pattern4_invalid.match(txt):
        return False
    else:
        return True

In [6]:
def find_questions(text):

    question_tuples = []
    for i in range(len(text)):
        pattern1 = re.findall('(N*Q[0-9]+_*[A-Za-z]*)(.*)', text[i])
        pattern2 = re.findall('(NA-[0-9]+)(.*)', text[i])
        if pattern1:
            question = pattern1[0][1]
            for j in range(1,1000):
                if i+j < len(text):
                    if is_valid(text[i+j]):
                        if text[i+j]:
                            question += text[i+j]
                    else:
                        break
                else:
                    break
            num = pattern1[0][0]
            pattern_cleaned = (num, question)
            question_tuples.append(pattern_cleaned)
        if pattern2:
            num = pattern2[0][0]
            question = pattern2[0][1]
            pattern_cleaned = (num, question)
            question_tuples.append(pattern_cleaned)
    return question_tuples

In [7]:
question_tuples = find_questions(text)

In [8]:
question_tuples = question_tuples[0:-1]

In [10]:
def to_csv(tuple_list, name):
    cleaned_data = pd.DataFrame()
    for tup in question_tuples:
        var_name = tup[0]
        question = tup[1]
        cleaned_data = cleaned_data.append([[var_name, question]], ignore_index=True)
    cleaned_data.columns = ["Var_Name", "Var_Text"]
    cleaned_data.to_csv(path_or_buf=name + ".csv", index=False)

In [12]:
to_csv(question_tuples, "2016CCSLeadershipSurvey")